## Math model scripts


### first we create a generating function to keep track of the probability that the number of approvals = k
### the generating function is in the form:
$g(x) = p_a(x)^1 + p_r(x)^-1 + p_i(x)^0$
for a single approval

for n many agents sending an approval, the generating function looks like:
$g(x)^n = (p_a(x)^1 + p_r(x)^-1 + p_i(x)^0)^n$

In [1]:
import sympy as sp
#we will manipulate the generating function symbolically to compute the probability of success in a single depth level for a single parent
import math
import numpy as np

In [3]:
def generating_function(x:sp.Symbol, p_a:sp.Symbol, p_r:sp.Symbol, p_i:sp.Symbol, n:sp.Symbol)->sp.Expr:

    g = (p_a*(x**1) + p_r*(x**-1) + p_i*(x**0))**n
    return g

In [5]:
x, n = sp.symbols('x, n')
p_a, p_r, p_i = sp.symbols('p_a, p_r, p_i')

g = generating_function(x, p_a, p_r, p_i, n)
print(type(g))

<class 'sympy.core.power.Pow'>


In [10]:
#here we will sub into n the number of neighbours a parent has at depth d

def sub_neighbours_into_g(g:sp.Expr, n_symbolic:sp.Symbol, n:int):
    gn = g.subs(n_symbolic, n)
    expanded_gn = sp.expand(gn)
    return expanded_gn

In [ ]:
def coefficients_g(expanded_generating_function:sp.Expr, x:sp.Symbol)-> dict:
    """extracting coefficients of x from the expanded generating function.
    expanded_generating_function: expanded generating function: sympy expression
    x: variable to extract coefficients from. 

    returns: dict:{power: coefficient}
    """
    # Dictionary to hold coefficients by power
    coeff_dict = {}

    # Break the expression into terms and analyze powers
    for term in expanded_generating_function.as_ordered_terms():
        term = sp.expand(term)
        coeff, power = term.as_coeff_exponent(x)
        coeff_dict[power] = coeff_dict.get(power, 0) + coeff

    return coeff_dict

In [ ]:
def prob_success_single_parent_at_depth_d(coeff_dict:dict, n:int, k:int, p_accept:float, p_reject:float, p_ignore:float):
    """probability of a single parent receiving enough approvals at a given depth level d"""
    #range(np.ceil(n_d*t), n_d +1)
    prob_success = 0
    for exponent in range(k, n+1):

        prob_success += coeff_dict[exponent].subs({p_a: p_accept, p_r: p_reject, p_i: p_ignore, x: 1})
    return prob_success


In [ ]:
def binomial_probability(p_success, n, k):
        n_choice_k = math.factorial(n)/(math.factorial(k) * math.factorial(n-k))
        binomial_prob = n_choice_k * ((p_success ** k) * ((1-p_success) ** (n-k)))
        return binomial_prob

In [ ]:
def probability_of_success_at_depth_d(p_success:float, N:int, t:float)->float:
    """p_s: probability of ENOUGH parents receiving enough approvals from their n_d children
    N: number of parents in that depth level
    t: threshold
    
    returns: probability of enough PARENTS succeeding at depth level d"""
    k = int(np.ceil(N*t))

    for i in range(N, k):

        # Calculate the binomial probability
        prob_sucess_depth = 0

        #summing over all the possible valid outcomes. (ie: from getting k approvals, to k+1, k+2, ... n)
        for i in range(k, N+1):
            #print(i)
            binomial_prob= binomial_probability(p_success, n, i)
            prob_sucess_depth += binomial_prob
    return prob_sucess_depth


In [ ]:
def prob_TCA_True(height, branching_factor, threshold):
    total_prob_sucess = 1
    


In [ ]:
height = 3
branching_factor = [2,3,4,0]
threshold = 1

x, symbolic_n = sp.symbols('x, n')
p_a, p_r, p_i = sp.symbols('p_a, p_r, p_i')

#TODO: placeholder, in simulator will feed the values of the probabilities from the for loop.
prob_accept, prob_reject, prob_ignore = 0.5, 0.3, 0.2

for d in range(height-1, -1, -1):
        n = branching_factor[d]
        k = int(np.ceil(branching_factor[d] * threshold))
        #print(d, n)
        
        #NOTE here p_a, p_r, p_i are symbolic sympy variables
        g = generating_function(x, p_a, p_r, p_i, symbolic_n)
        expanded_g = sub_neighbours_into_g(g, symbolic_n, n)
        coeffs_dict = coefficients_g(expanded_g, x)

        #NOTE here prob_accept, prob_reject, prob_ignore are literal float values
        p_parent_success = prob_success_single_parent_at_depth_d(coeffs_dict, n, k, prob_accept, prob_reject, prob_ignore)

        p_A_d = probability_of_success_at_depth_d(p_parent_success, N, threshold)
        

2 4
1 3
0 2
